In [13]:
import pandas as pd
from pathlib import Path
import re
from pandas.api.types import CategoricalDtype

# All SOFR swap rate files
# Folder with the Excel files (repo root / swap_data)
repo_root = Path.cwd()
folder = repo_root / "swap_data"

files = sorted(folder.glob("*.xlsx"))

rows = []

for f in files:
    # Example expected filenames:
    # "USD SOFR 1W Historical Swap Rates - BlueGamma.xlsx"
    # "USD SOFR 34M Historical Swap Rates - BlueGamma.xlsx"
    m = re.search(r"SOFR\s+(\d+[DWMY])\s*Historical", f.name)
    if not m:
        # Skip files that don't look like SOFR tenor files
        continue

    tenor = m.group(1)

    df = pd.read_excel(f, usecols=["Date", "Rate"])
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df["Tenor"] = tenor

    rows.append(df)

if not rows:
    raise ValueError(
        "No tenor files matched the regex. "
        "Check the filename pattern and the regex."
    )

panel_long = pd.concat(rows, ignore_index=True)
panel_long = panel_long[["Date", "Tenor", "Rate"]]

# Tenor ordering: 1W, 2W, 3W, 1M, ..., 35M, 3Y, ..., 50Y
weeks = [f"{i}W" for i in range(1, 4)]
months = [f"{i}M" for i in range(1, 36)]
years = [f"{i}Y" for i in range(3, 51)]
tenor_order = weeks + months + years

tenor_cat_type = CategoricalDtype(categories=tenor_order, ordered=True)
panel_long["Tenor"] = panel_long["Tenor"].astype(tenor_cat_type)

panel_long = (
    panel_long
    .sort_values(["Date", "Tenor"])
    .reset_index(drop=True)
)

# We need to keep only one observation for the last date, since is duplicated 
# find the last date 
last_date = panel_long["Date"].max()

# keep the first 
mask = panel_long["Date"] == last_date
clean_last = panel_long[mask].drop_duplicates(subset=["Date", "Tenor"], keep="first")

# 3. Ricostruisco il dataframe senza i duplicati
panel_long = pd.concat([
    panel_long[panel_long["Date"] != last_date],  # tutte le altre date
    clean_last                                    # ultima data ripulita
], ignore_index=True)

# 4. Riordino
panel_long = panel_long.sort_values(["Date", "Tenor"]).reset_index(drop=True)

out_path = repo_root / "SOFR_panel_Date_Tenor_Rate_sorted.csv"
panel_long.to_csv(out_path, index=False)

def sofr_curve(date_):
    """
    Return the SOFR curve (Tenor, Rate) for a given date.
    `date_` can be 'YYYY-MM-DD', datetime.date, or pd.Timestamp.
    """
    if isinstance(date_, str):
        date_ = pd.to_datetime(date_).date()
    elif isinstance(date_, pd.Timestamp):
        date_ = date_.date()
    return panel_long[panel_long["Date"] == date_].reset_index(drop=True)


In [14]:
import numpy as np
np.unique(df["Date"])

array([datetime.date(2025, 10, 17), datetime.date(2025, 10, 20),
       datetime.date(2025, 10, 21), datetime.date(2025, 10, 22),
       datetime.date(2025, 10, 23), datetime.date(2025, 10, 24),
       datetime.date(2025, 10, 27), datetime.date(2025, 10, 28),
       datetime.date(2025, 10, 29), datetime.date(2025, 10, 30),
       datetime.date(2025, 10, 31), datetime.date(2025, 11, 3),
       datetime.date(2025, 11, 4), datetime.date(2025, 11, 5),
       datetime.date(2025, 11, 6), datetime.date(2025, 11, 7),
       datetime.date(2025, 11, 10), datetime.date(2025, 11, 12),
       datetime.date(2025, 11, 13), datetime.date(2025, 11, 14)],
      dtype=object)

In [15]:
sofr_curve("2025-11-14")

,Date,Tenor,Rate
0,2025-11-14,1W,3.984740
1,2025-11-14,2W,3.975849
2,2025-11-14,3W,3.984226
3,2025-11-14,1M,3.975123
4,2025-11-14,2M,3.935190
...,...,...,...
81,2025-11-14,46Y,3.847796
82,2025-11-14,47Y,3.835870
83,2025-11-14,48Y,3.823154
84,2025-11-14,49Y,3.809541
